In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# -------------------
# 1. 데이터 불러오기
# -------------------
df = pd.read_csv("../../data/TRA-TRAACCIDENT/TRA-TRAACCIDENT.csv")

# -------------------
# 2. 소계 제거
# -------------------
df = df[df["자치구"] != "소계"].copy()

# -------------------
# 3. 필요한 컬럼 선택
# -------------------
df["발생건수"] = pd.to_numeric(df["발생건수 (건)"], errors="coerce")
df["사망자수"] = pd.to_numeric(df["사망자수 (명)"], errors="coerce")
df["부상자수"] = pd.to_numeric(df["부상자수 (명)"], errors="coerce")

# -------------------
# 4. 위험도 점수 계산(사고가 많을수록 점수가 높음))
# -------------------
scaler = MinMaxScaler(feature_range=(0, 100))
scaled_values = scaler.fit_transform(df[["발생건수", "사망자수", "부상자수"]])

df[["traaccident_count_score", "traaccident_death_score", "traaccident_injury_score"]] = scaled_values

# 최종 위험도 점수 (평균)
df["traaccident_score"] = df[[
    "traaccident_count_score",
    "traaccident_death_score",
    "traaccident_injury_score"
]].mean(axis=1)

# -------------------
# 5. 필요한 결과만 정리
# -------------------
result = df[["자치구", "발생건수", "사망자수", "부상자수", "traaccident_score"]].copy()
result.rename(columns={"자치구": "gu"}, inplace=True)

# -------------------
# 6. CSV 저장
# -------------------
result.to_csv("../../result/TRA/TRA-TRAACCIDENT_result.csv", index=False, encoding="utf-8-sig")

print(result.head())

    gu  발생건수  사망자수  부상자수  traaccident_score
1  종로구   917     5  1253          14.257263
2   중구   976    16  1236          35.100954
3  용산구   868    12  1174          26.086682
4  성동구   919     7  1155          17.202332
5  광진구   785     4  1018           9.175386
